In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1842,2024-06-28,Eua Wnba,23:00,Phoenix Mercury F,Los Angeles Sparks F,1.20,4.60,164.5,1.81,1.95,0.0,0.00,0.00,na3yJs3D,0.833333,0.217391,0.552486,0.512821,0.050725,257.954,51.010754,0.197751,1.8,1.643168,0.912871,323.73,3.060,0.616847,0.201584,-6.0,458.704,167.899393,0.366030,0.0,0.000000,NaN,608.00,6.036,2.148600,0.355964,-56.0,99,80,3.27,7.60,236.388,458.704,0.829022,0.052657,NaN,5.10,1.020,0.196078,0.000000,0.0,0.000000,-5.00,-1.000,-3.600000,0.000000,0.0,0.000000
1843,2024-06-28,Indonésia Ibl,08:00,Kesatria Bengawan Solo,Prawira Bandung,2.48,1.50,164.5,1.85,1.85,0.0,0.00,0.00,GSURptZh,0.403226,0.666667,0.540541,0.540541,0.069892,217.738,118.486550,0.544170,1.8,1.643168,0.912871,111.10,2.400,1.121873,0.467447,-31.0,89.386,8.058457,0.090153,3.0,0.000000,0.000000,83.43,1.060,0.059161,0.055812,98.0,101,81,1.10,1.03,129.704,164.634,0.348223,0.000000,NaN,-0.11,-0.022,-67.272727,0.000000,0.0,0.000000,1.88,0.376,1.329787,0.000000,0.0,0.000000
1844,2024-06-28,Indonésia Ibl,08:00,Pacific Caesar Surabaya,Pelita Jaya,11.70,1.01,169.5,1.83,1.87,0.0,0.00,0.00,CfOzqK35,0.085470,0.990099,0.546448,0.534759,0.075569,322.702,117.251727,0.363344,0.6,1.341641,2.236068,327.99,4.606,2.841290,0.616867,-66.0,126.166,65.601973,0.519966,3.0,0.000000,0.000000,115.28,1.376,0.548389,0.398538,44.0,87,88,3.77,1.31,511.466,124.350,1.189453,0.015289,NaN,-5.00,-1.000,-10.700000,0.000000,0.0,0.000000,1.73,0.346,0.028902,0.000000,0.0,0.000000
1845,2024-06-28,Mundo Amistoso Internacional,12:30,Finlândia,Letônia,2.97,1.36,159.5,1.87,1.83,0.0,0.00,0.00,MP1S5SCl,0.336700,0.735294,0.534759,0.546448,0.071994,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,73,84,1.57,1.10,0.000,0.000,0.525839,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1846,2024-06-28,Mundo Amistoso Internacional,13:00,Polónia,Nova Zelândia,1.34,3.08,157.5,1.85,1.85,0.0,0.00,0.00,lMrLS6a3,0.746269,0.324675,0.540541,0.540541,0.070944,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,73,70,3.53,2.31,0.000,0.000,0.556727,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1847,2024-06-28,Mundo Amistoso Internacional,14:00,Espanha,República Dominicana,1.20,4.22,158.5,1.84,1.86,0.0,0.00,0.00,Emhc6Y4M,0.833333,0.236967,0.543478,0.537634,0.070300,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,84,80,1.36,1.23,0.000,0.000,0.787994,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1848,2024-06-28,Mundo Amistoso Internacional,15:00,Eslovénia,Brasil,1.51,2.45,169.5,1.87,1.79,0.0,0.00,0.00,p80tAG5c,0.662252,0.408163,0.534759,0.558659,0.070415,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,70,81,2.45,2.07,0.000,0.000,0.335697,0.030912,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1849,2024-06-28,Nova Zelândia Nbl,04:30,Taranaki Ai

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1851,19:45,Venezuela Superliga,Piratas de La Guaira,Guaiqueries,2.06,Back Home
1872,04:30,Austrália Nbl1 Sul,Launceston F,Diamond Valley F,1.54,Back Home
